# Project - Part 1

### Libraries import:

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import pandas as pd
import json
import re

#### This variables definitions will help us later

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
car_type = str(input("Which car model data would you like to get? "))

Which car model data would you like to get? אאודי


### Function for getting all specific cars Id's from given soup:

In [3]:
def get_model_links(soup):
    desired_links = []
    for car in soup('div',class_="card-body p-md-3"):
        s_link = car.find('a').get('href')
        desired_links.append(s_link)
    return desired_links [:-3]

### A loop that gets relevant urls (using `get_model_links` function) and insert them into one list:
When `ads` will contain empty list - there are no more pages - the loop will break.

In [4]:
main_audi_url = 'https://www.ad.co.il/car?sp261=13906&pageindex=' # Pages of all Audis
page = 1
audis_links = []
while True:  # Runnning along pages
    idx_url = main_audi_url + str(page)
    html = requests.get(idx_url)
    content = html.content
    soup = BeautifulSoup(content, 'html.parser')
    ads = get_model_links(soup) # Using previous function
    if len(ads) != 0:
        audis_links.extend(ads) # Our previous function
        page += 1 # Move to the next page
    else:
        break

In [5]:
print("Amount of requested cars: ", len(audis_links), "\n")

Amount of requested cars:  118 



### Running along Audis ads and scrap relevant information:
The loop attaching the `main_url` to extracted urls from `audis_links` list.
Inside the loop written fixed commands that scrap desired information about each car and save it in `car_att_dict` - attributes dictionaries that at the end combined to one big list `all_cars_info`.

In [6]:
all_cars_info = []
main_url = "https://www.ad.co.il"
for identifier in audis_links:
    full_url = main_url + identifier
    try:
        html = requests.get(full_url, headers)
        content = html.content
        soup = BeautifulSoup(content, 'html.parser')
        table = soup.find('div', class_ = "col-xxl-4 col-lg-4 col-md-5")
        table2 = soup.find('div', class_ = "col-xxl-8 col-lg-8 col-md-7")

        car_att_dict = {}
        car_att_dict['Car_name'] = table.find('div',class_ = "d-flex justify-content-between")('h2')[0].get_text()

        try:
            car_att_dict['Price'] = table.find('div',class_ = "d-flex justify-content-between")('h2')[1].get_text()
        except IndexError:
            car_att_dict['Price'] = None

        for a in table.find_all('tr'):  # Loop for extraction of relevant attributes and puttnig them in small dictionary for every car.
            key = a.find_all('td')[0].get_text().strip()
            value = a.find_all('td')[1].get_text().strip() 
            car_att_dict[key] = value

        car_att_dict['Pic_num'] = len(table2('figure'))
        
        try:
            car_att_dict['Description'] = table2.find('p').get_text().strip()
        except AttributeError:
            car_att_dict['Description'] = None
            
        car_att_dict['Cre_date'] = table2.find_all('div', 'px-3')[0].get_text()[-10:]
        car_att_dict['Repub_date'] = table2.find_all('div', 'px-3')[1].get_text()[-10:]
        
        all_cars_info.append(car_att_dict)
        
    except:
        print("Invalid URL, please check and try again")
    
    

### Push our product into `pd.DataFrame`:

In [7]:
data = pd.DataFrame(all_cars_info)

### Rename columns:

In [8]:
# Renaming:
data.rename(columns = {'יד': 'Hand',
                       'שנה': 'Year',
                       'ת. הילוכים': 'Gear',
                       'נפח': 'capacity_Engine',
                       'סוג מנוע': 'Engine_type',
                       'ק"מ': 'Km',
                       'צבע': 'Color',
                       'בעלות קודמת': 'Prev_ownership',
                       'בעלות נוכחית': 'Curr_ownership',
                       'אזור': 'Area',
                       'עיר': 'City',
                       'טסט עד': 'Test'},
              inplace=True)

### Cast and manipulate:

In [9]:
data['Cre_date'] = pd.to_datetime(data['Cre_date'], dayfirst = True)
data['Repub_date'] = pd.to_datetime(data['Repub_date'], dayfirst = True)
data['Gear'] = pd.Categorical(data['Gear'])
data['Engine_type'] = pd.Categorical(data['Engine_type'])
data['Prev_ownership'] = pd.Categorical(data['Prev_ownership'])
data['Curr_ownership'] = pd.Categorical(data['Curr_ownership'])
data['Year'] = data['Year'].astype('Int32')
data['Hand'] = data['Hand'].astype('Int32')
data['Pic_num'] = data['Pic_num'].astype('Int32')
data['capacity_Engine'] = data['capacity_Engine'].str.replace(',', '').astype('Int32')
data['Km'] = data['Km'].str.replace(',', '').astype('Int32')
data['Price'] = data['Price'].str.replace(r'[^\d]', '', regex = True).astype('float') # Extract only the digits using regular expressions

### `Test` column handling:
Calculate the time difference between today and future test expire date.

In [10]:
today = pd.to_datetime('today')
data['Test'] = data['Test'].apply(lambda x: "30/" + x if pd.notna(x) else x) # Test time is at the end of the month
data['Test'] = pd.to_datetime(data['Test'], dayfirst = True)
data['Test'] = (data['Test'] - today).dt.days.astype('Int32')

### Renaming for identical column names - will be helpful for supply score manipulation


In [11]:
data = data.rename(columns={'Car_type': 'Type'})

### Additional data cleaning:
* Fake ads need to be be removed!

In [12]:
fakes = data[data['Description'].str.contains('דרושים|משרד', na=False)]
fakes['Description']

92     לעבודה מהבית דרושים דרושות, להקלדה, הזנת נתוני...
101    דרושים דרושות לעבודה מהבית בקלדנות, כתיבה, הזנ...
109                                         עבודה משרדית
112    דרושים/ות לעבודה בהקלדת נתונים, מדובר על עבודה...
113    דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...
114    דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...
115    דרושים/ות לעבודה מהבית, (גברים ונשים) בסוכנות ...
116    דרושים דרושות לעבודה מהבית בקלדנות, כתיבה, הזנ...
117    דרושים! לעבודה באקסל, מדובר על עבודה בהזנת תוכ...
Name: Description, dtype: object

In [13]:
# Everything except fakes
data = data[~data['Description'].str.contains('דרושים|משרד', na=False)]

---

## Supply score integration:

In [14]:
url = "https://data.gov.il/api/3/action/datastore_search?resource_id=5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6"
response = requests.get(url)
try:
    from_gov = json.loads(response.content)
    from_gov = from_gov['result']['records']
except:
    print("Data in update progress, please try again later")

In [15]:
pd.DataFrame(from_gov).sample(5)

,_id,sug_degem,tozeret_cd,tozeret_nm,tozeret_eretz_nm,tozar,degem_cd,degem_nm,shnat_yitzur,mispar_rechavim_pailim,mispar_rechavim_le_pailim,kinuy_mishari
23470,23471,P,650,סובארו יפן,יפן,סובארו,400,GDACL8D,2003,7,11,IMPREZA TUR NEW
5727,5728,P,413,טויוטה יפן,יפן,טויוטה,1001,ZVW30L-AHXEBW,2009,640,90,PRIUS HYBRID
12536,12537,P,430,טויוטה אנגליה,בריטניה,טויוטה,47,ZWE186L-DHXNBW,2018,5,1,AURIS HYBRID
14532,14533,P,590,מיצובישי יפן,יפן,מיצובישי,130,N43WLNUEL6,1998,0,5,SPACE WAGON
342,343,P,155,הונדה טורקיה,טורקיה,הונדה,81,FD76,2008,3447,1302,CIVIC


### Create a Dataframe from the API data for more convinient manipulation:

In [16]:
temp_list = []
for item in from_gov:
    if item ['tozar'] == car_type: # filter all required car manufactor
        temp_dict = {}
        temp_dict['Type'] = item['kinuy_mishari']
        temp_dict['Year'] = item['shnat_yitzur']
        temp_dict['Supply_score'] = item['mispar_rechavim_pailim']       
        temp_list.append(temp_dict)
data2 = pd.DataFrame(temp_list)
data2

,Type,Year,Supply_score
0,A3 SEDAN,2019,30
1,A6,2006,13
2,A4,2003,3
3,A6,2004,6
4,TT,2001,3
...,...,...,...
1607,AUDI A3,2010,14
1608,A7,2014,13
1609,A6,2006,4
1610,ALLROAD,2001,4


### Display the discrepancy between names:
We can notice the two sources' names are not similar 

In [17]:
types = data.Car_name.unique()
types2 = data2.Type.unique()

print('Our models: \n',types, '\n')
print('External models:\n ', types2)

Our models: 
 ['אאודי A5' 'אאודי A1' 'אאודי RS5' 'אאודי A6' 'אאודי A3' 'אאודי 100'
 'אאודי TT' 'אאודי A4' 'אאודי Q3' 'אאודי RS3' 'אאודי A7' 'אאודי S3'
 'אאודי S5' 'אאודי TT קופה' 'אאודי R8' 'אאודי 80' 'אאודי S8' 'אאודי S7'] 

External models:
  ['A3 SEDAN' 'A6' 'A4' 'TT' 'Q7' 'A8' 'Q5' 'AUDI TT' 'AUDI Q7' 'A1'
 'RSQ3 SPORTBACK' 'A5' 'RS5' 'AUDI S3' "אודי 6-Aפרטי רג' 4דל" 'Q3' 'A3'
 'אוטומטי' 'A3 DESIGN' 'A1 SPORTBACK' 'RSQ3 PERFORMANC' 'A3 SPORTBACK'
 'A6 2.8Q' 'S5 SPORTBACK' 'SPORTBACK A3' 'A7' 'TT ROADSTER'
 'RS3 SPORTBACK' 'S3' 'Q5 DESIGN' 'A5 COUPE' 'Q2' 'RS3' 'A1 SPORTBCK' 'Q8'
 'SQ5 SPORTBACK' 'S8' 'Q3 SPORTBACK' 'S3 SPORTBACK' 'AUDI A4' 'TT COUPE'
 'S5' 'SQ7' 'RSQ3' 'AUDI A5' 'A3 CABRIOLET' "אודי 8A 7.3 אוט'WEA"
 'RS3 SEDAN' 'S1' "אודי 8A V6 4דלת' 8.2" 'ALLROAD' '5 הילוכים'
 'A5 SPORTBACK' 'אודי 3A 8.1 אוטומטי' 'ETRON SPORTBACK'
 "אודי 4A 4דלת' T8.1" 'Q5 SPORTBACK' 'S4' 'RODSTR TT' 'אוטומטי 4 הילוכים'
 'Q7 E-TRON' 'Q5 TDI' 'S6' 'A7 SPORTBACK' 'Q7 50TDI' 'A3 SB E-TRON'
 'RS5 COUP

### Function for increasing the similarity between the datasets using regular expressions:


In [18]:
def extract_simple_models(models):
    simple_models = []
    for model in models:
        match = re.match(r'([A-Za-z]+\d*)', model)
        if match:
            simple_models.append(match.group())
        else:
            simple_models.append(None)
    return simple_models

### Extracting unique simple car models using `extract_simple_models` function
 and changing the car model names to more simple names

In [19]:
unique_simple_models = extract_simple_models(data2.Type)
data2['Type'] = unique_simple_models

### Fixing specific minor inconsistencies:

In [20]:
data['Type'] = data['Car_name'].str.replace(car_type+" ", '')
data['Type'] = data['Type'].str.replace(" קופה", "")
data['Type'] = data['Type'].str.replace("All Road", "ALLROAD")
data['Type'] = data['Type'].str.replace(" ", "")

### Repeated comparison:

In [21]:
types = data.Type.unique()
types2 = data2.Type.unique()

print('Our models: \n',types, '\n')
print('External models:\n ', types2)

Our models: 
 ['A5' 'A1' 'RS5' 'A6' 'A3' '100' 'TT' 'A4' 'Q3' 'RS3' 'A7' 'S3' 'S5' 'R8'
 '80' 'S8' 'S7'] 

External models:
  ['A3' 'A6' 'A4' 'TT' 'Q7' 'A8' 'Q5' 'AUDI' 'A1' 'RSQ3' 'A5' 'RS5' None
 'Q3' 'S5' 'SPORTBACK' 'A7' 'RS3' 'S3' 'Q2' 'Q8' 'SQ5' 'S8' 'SQ7' 'S1'
 'ALLROAD' 'ETRON' 'S4' 'RODSTR' 'S6' 'Q4' 'QUATTRO' 'RS6' 'E' 'TTS' 'RS7'
 'S7' 'TTRS']


### Representing supply score for each car model by years:

In [22]:
grouped = data2.groupby(['Type','Year']).sum().reset_index()
grouped

,Type,Year,Supply_score
0,A1,2010,31
1,A1,2011,283
2,A1,2012,312
3,A1,2013,268
4,A1,2014,204
...,...,...,...
301,TT,2020,55
302,TT,2021,62
303,TT,2022,4
304,TTRS,2022,0


### Final data preparation:

In [23]:
# merging with the grouped data for getting the supply score per model per year
new_data = pd.merge(data, grouped, on=['Type', 'Year'], how='left')
new_data['Supply_score'] = new_data['Supply_score'].astype('Int32')

In [24]:
new_data['model'] = new_data['Type']
new_data['manufactor'] = new_data['Car_name']  
new_data['manufactor'] = new_data['manufactor'].apply(lambda x: car_type)
new_data.drop(['Type','Car_name'], inplace = True ,axis=1)

In [25]:
# Columns sorting
columns_order = ['manufactor', 'Year', 'model', 'Hand', 'Gear',  'capacity_Engine', 'Engine_type', 'Prev_ownership', 'Curr_ownership', 'Area', 'City', 'Price', 'Pic_num', 'Cre_date', 'Repub_date', 'Description', 'Color', 'Km', 'Test', 'Supply_score']
new_data = new_data[columns_order]

### Final data:

In [26]:
new_data

,manufactor,Year,model,Hand,Gear,capacity_Engine,Engine_type,Prev_ownership,Curr_ownership,Area,City,Price,Pic_num,Cre_date,Repub_date,Description,Color,Km,Test,Supply_score
0,אאודי,2013,A5,3,אוטומטית,1800,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,68000.0,5,2024-03-17,2024-03-31,‎!!קילומטר נמוך !! רכב במצב תצוגה טיפול גדול ח...,שחור,119000,<NA>,29
1,אאודי,2016,A1,2,אוטומטית,1000,בנזין,ליסינג,פרטית,NaN,בית ג'ן,55000.0,3,2023-11-25,2023-11-24,רכב שמור מטופל בזמן יש עליו כל מיני תוספות מי ...,שחור,135000,<NA>,344
2,אאודי,2021,A1,1,אוטומטית,1000,בנזין,פרטית,פרטית,ירושלים והסביבה,ירושלים,122000.0,9,2023-11-11,2023-11-11,רכב אאודי A1 דגם סטייל קומפורט \nאחרי טיפול של...,שחור,<NA>,<NA>,254
3,אאודי,2019,RS5,4,אוטומטית,4500,גז,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,65000.0,4,2023-10-02,2023-10-02,אאודי RS5 ספורטבק 2019 כמעט חדשה בדייטונה גריי...,אפור,7000,<NA>,4
4,אאודי,2019,RS5,4,אוטומטית,4500,גז,פרטית,פרטית,חולון - בת ים,חולון,65000.0,4,2023-10-02,2023-10-02,נתניהאאודי RS5 ספורטבק 2019 כמעט חדשה בדייטונה...,אפור,7000,<NA>,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,אאודי,2016,A3,2,אוטומטית,1400,בנזין,פרטית,פרטית,חיפה וחוף הכרמל,טירת כרמל,93000.0,9,2022-01-27,2022-01-27,רכב שמור ללא תיקון צבע כל התוספות חסכוני אמין ...,לבן פנינה,110000,<NA>,590
105,אאודי,2016,A6,2,טיפטרוניק,2000,בנזין,פרטית,פרטית,NaN,עראבה,165000.0,0,2022-06-12,2022-06-12,אודי a6 לקצרי. מטופלת בקנאות שמורה מאוד. חיצונ...,לבן מטאלי,120000,<NA>,70
106,אאודי,2009,A4,2,טיפטרוניק,1800,בנזין,NaN,NaN,באר שבע והסביבה,אופקים,25000.0,0,2022-06-10,2022-06-10,אאודי שמורה מאוד מצב חדש \r\nמתחייב בבדיקה מלא...,NaN,200000,<NA>,171
107,אאודי,2015,A6,3,אוטומטית,3000,בנזין,פרטית,פרטית,נתניה והסביבה,נתניה,155000.0,0,2022-05-01,2022-05-01,Alex,אדום,90000,<NA>,89


---

### Write to csv

In [27]:
file_path = r"C:\Users\elon2\OneDrive\Desktop"
csv_path = os.path.join(file_path, "Audis.csv")
new_data.to_csv(csv_path, index=False, encoding='utf-8-sig')

<IPython.core.display.Javascript object>